In [4]:
import os
import numpy as np
from pathlib import Path
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch

image_dir_path = 'dataset'
paths = []
pokemonNames, pokemonTarget, pokemonRGB, pokemonImage = [], [], [], []

def createDataDebug(pokemon_bunch_data):
    print(pokemon_bunch_data['data'].shape)
    print(pokemon_bunch_data['images'].shape)
    print(pokemon_bunch_data['target'].shape)
    print(pokemon_bunch_data['target_names'].shape)

def img_normalize():
    for pokeFolder in os.listdir(image_dir_path):
        for i, imageFile in enumerate(os.listdir(f"{image_dir_path}/{pokeFolder}")):
            image_path = f"{image_dir_path}/{pokeFolder}/{imageFile}"
            if imageFile[imageFile.index('.'):].lower() in ['.png', '.jpg', '.jpeg']:
                img = Image.open(image_path)
                img.resize((128,128)).convert('RGB').save(image_path)
                if not (imageFile.startswith("new_img")) :
                    os.rename(image_path, f"{image_dir_path}/{pokeFolder}/new_img{i}{imageFile[imageFile.index('.'):]}")
            else:
                os.remove(image_path)

img_normalize()

def createData():
    for path in Path(image_dir_path).rglob('*.*'):
        if path.parts[1] not in pokemonNames:
            pokemonNames.append(path.parts[1])
        pokemonTarget.append(pokemonNames.index(path.parts[1]))
        img = Image.open(path)
        pixel = img.load()
        tempRGB, tempRow = [], []
        for i in range(img.height):
            tempImage = []
            for j in range(img.width):
                tempRGB.extend([pixel[i, j][0], pixel[i, j][1], pixel[i, j][2]])
                tempImage.append(np.asarray(pixel[i, j]))
            tempRow.append(tempImage)
        pokemonImage.append(tempRow)
        pokemonRGB.append(tempRGB)

    return Bunch( data=np.asarray(pokemonRGB), 
                  images=np.asarray(pokemonImage),
                  target=np.asarray(pokemonTarget),
                  target_names=np.asarray(pokemonNames) )

pokemon_bunch_data = createData()

pokeTrain, pokeTest, labelTrain, labelTest = \
    train_test_split(pokemon_bunch_data['data'], pokemon_bunch_data['target'], test_size=0.1)

# # # ########### Random Forest Algorithm ###########

forest = RandomForestClassifier(n_estimators=100).fit(pokeTrain, labelTrain)
print("Now Testing RandomForest: \n " + "Accuracy: " + str(forest.score(pokeTest, labelTest)) + "%\n")

# # # ########### KNearestNeighbors Algorithm ###########

pokemonKNN = KNeighborsClassifier(n_neighbors = 3).fit(pokeTrain, labelTrain)
print("Now Testing kNN: \n " + "Accuracy: " +  str(pokemonKNN.score(pokeTest, labelTest))+ "%")

# All 150 pokemon takes ~40 minutes are so // Before Refactor


Now Testing RandomForest: 
 Accuracy: 0.9080459770114943%

Now Testing kNN: 
 Accuracy: 0.7931034482758621%
